In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk

## User Input

In [2]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")
    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver


## Scraping

In [3]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(user_url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("User does not exist")

working


### Profile Info

In [4]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''

post_karma_box = soup.find('span', {'class': 'karma'})
post_karma = post_karma_box.text if post_karma_box is not None else ''

data = {'username': username, 'comment_karma': comment_karma, 'post_karma': post_karma}

json_data = json.dumps(data)

print(json_data)

{"username": "NPU-F", "comment_karma": "14,739", "post_karma": "27,091"}


### User Posts & Comments

In [ ]:
# JSON FORMAT:
#     Username:
#         Post Karma:
#         Comment Karma:
#         Account Creation Date:
#         Number of Submissions:
#         Number of COmments:
#     Posts:
#         utc_date:
#         Type:
#         Subreddit:
#         Title:
#         File_Path:
#         Permalink:
#         Score:
#     Comments:
#         utc_date:
#         Subreddit:
#         Text:
#         Score:
#         Permalink:
    

In [27]:
comments = []
for comment in soup.find_all('div', class_='comment')[:1]:
    for ps in comment:
        print(ps.text)

Theft of ATL airport funds by airport executives (corrected from the AJC) by [deleted] in Atlanta

[–]NPU-F 31 points32 points33 points 3 days ago (0 children)Keisha Lance Bottoms made up a job at the airport for her campaign manager

Marva Lewis, Bottoms’ campaign manager and one of the six people placed on the city payroll before Bottoms took office, was given the title of deputy general manager at the airport with a salary of $273,873 per year and never worked at the airport. Lewis, who became Bottoms’ chief of staff, held the airport job classification while leading the transition and for two weeks after the new mayor was sworn into office.


permalinksavecontextfull comments (19)reportgive award




In [8]:
posts = []
for post in soup.find_all('div', class_='link'):
    print("2")
    #

2
2
2
2
2
2
2
2
2
2
2
2


In [ ]:
user_data = {
    'username': username,
    'karma': {
        'post_karma': post_karma,
        'comment_karma': comment_karma
    },
    'posts': posts,
    'comments': comments
}

**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 

**TO-DO PT3**
* 